# Import Required Libraries
Import the necessary libraries, including pandas, numpy, and sklearn.

In [1]:
import os
os.chdir('../../')

In [2]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

# Load Test Dataset
Load the dataset `case/data/processed/test_dataset_with_target.parquet`, which contains the target variable `default`.

In [3]:
# Load Test Dataset
test_dataset = pd.read_parquet('case/data/processed/test_dataset_with_target.parquet')

# Verificar as primeiras linhas do dataset carregado
test_dataset.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,default
472065,130607974,14100,14100,14096.0,36,0.238647,552.00,E,E2,Sr Clinical Research Associate,...,NaT,NaT,-1,-1,NaN,-1.0,-1.0,-1.0,N,0.0
472066,130932509,20000,20000,20000.0,60,0.140747,466.00,C,C3,TEACHER,...,NaT,NaT,-1,-1,NaN,-1.0,-1.0,-1.0,N,1.0
472067,130938452,28625,28625,28624.0,60,0.218506,788.00,D,D5,Psychologist,...,NaT,NaT,-1,-1,NaN,-1.0,-1.0,-1.0,N,NaN
472068,130950844,20000,20000,20000.0,60,0.099182,424.25,B,B2,Manager,...,NaT,NaT,-1,-1,NaN,-1.0,-1.0,-1.0,N,NaN
472069,130966126,15000,15000,15000.0,60,0.174683,376.50,D,D1,CCAP Administrator,...,NaT,NaT,-1,-1,NaN,-1.0,-1.0,-1.0,N,NaN


In [46]:
test_dataset.shape

(538826, 142)

# Load Solution Datasets
Iterate through all datasets in the folder `case/data/solutions`. Each dataset can be in either Parquet or CSV format.

In [53]:
# Load Solution Datasets
import os

# Caminho para a pasta com os datasets de soluções
solutions_folder = 'case/data/solutions'

# Lista para armazenar os datasets de soluções
solution_datasets = []

# Iterar por todos os arquivos na pasta de soluções
for filename in os.listdir(solutions_folder):
    file_path = os.path.join(solutions_folder, filename)
    print(file_path)
    if filename.endswith('.parquet'):
        # Carregar dataset Parquet
        solution_dataset = pd.read_parquet(file_path)
    elif filename.endswith('.csv'):
        # Carregar dataset CSV
        solution_dataset = pd.read_csv(file_path)
    else:
        # Ignorar arquivos que não são Parquet ou CSV
        continue

    solution_dataset.columns = ['id', 'probabilities']
    # check if first row ID is equal to the test dataset
    if filename == 'jean_submission.parquet':
        solution_dataset['id'] = test_dataset['id'].values
    
    # Adicionar o nome do arquivo como uma coluna no dataset
    solution_dataset['solution_name'] = filename
    
    # Adicionar o dataset à lista
    solution_datasets.append(solution_dataset)
    print(filename, solution_dataset.shape)
    print('')

# Verificar as primeiras linhas do primeiro dataset de solução carregado
solution_datasets[0].head()

case/data/solutions/jean_submission.parquet
jean_submission.parquet (538826, 3)

case/data/solutions/henrique_schumann.csv
henrique_schumann.csv (538826, 3)

case/data/solutions/matheus_de_oliveira_alves_submission.parquet
matheus_de_oliveira_alves_submission.parquet (538826, 3)



,id,probabilities,solution_name
0,130607974,0.214025,jean_submission.parquet
1,130932509,0.304896,jean_submission.parquet
2,130938452,0.341226,jean_submission.parquet
3,130950844,0.209111,jean_submission.parquet
4,130966126,0.430642,jean_submission.parquet


# Join Datasets
For each solution dataset, join it with `test_dataset_with_target` on the column `id`.

In [54]:
# Join Datasets
joined_datasets = []

# Iterar por todos os datasets de soluções
for solution_dataset in solution_datasets:
    # Realizar o join com o dataset de teste usando a coluna 'id'
    joined_dataset = test_dataset.merge(solution_dataset, on='id')
    
    # Adicionar o dataset unido à lista
    joined_datasets.append(joined_dataset)

# Verificar as primeiras linhas do primeiro dataset unido
if joined_datasets:
    joined_datasets[0].head()

# Calculate Log Loss
Calculate the log loss for the target variable `default` using the prediction column `probabilities`.

In [56]:
# Calculate Log Loss
log_loss_results = []

# Iterar por todos os datasets unidos
for joined_dataset in joined_datasets:
    # Calcular o log loss para a variável alvo 'default' usando a coluna de predição 'probabilities'
    
    # Create a try catch block
    try:
        joined_dataset = joined_dataset[['probabilities', 'default', 'solution_name']].dropna()
        loss = log_loss(joined_dataset['default'], joined_dataset['probabilities'])
        solution_name = joined_dataset['solution_name'].iloc[0]
    except:
        loss = np.nan
        solution_name = 'error'
    
    # Adicionar o resultado à lista
    log_loss_results.append((solution_name, loss))

# Imprimir os resultados do log loss
for result in log_loss_results:
    print(f"{result[0]}, {result[1]:.4f}")

jean_submission.parquet, 0.5079
henrique_schumann.csv, 0.4861
matheus_de_oliveira_alves_submission.parquet, 0.4964


# Print Log Loss
Print the log_loss using the file name as the id name of the solution, with the following format: solution_name, log_loss.

In [ ]:
# Print Log Loss
for result in log_loss_results:
    print(f"{result[0]}, {result[1]:.4f}")